In [1]:
# get scripts for github
!git clone https://github.com/aashu-0/FineTuning_GPT2.git
%cd FineTuning_GPT2

Cloning into 'FineTuning_GPT2'...
remote: Enumerating objects: 177, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 177 (delta 92), reused 143 (delta 58), pack-reused 0 (from 0)
Receiving objects: 100% (177/177), 372.09 KiB | 12.00 MiB/s, done.
Resolving deltas: 100% (92/92), done.
/content/FineTuning_GPT2


In [2]:
# get current working directory
import os
print(os.getcwd())

/content/FineTuning_GPT2


In [3]:
!pip install torch>=2.1.0 \
            transformers>=4.36.2 \
            tokenizers>=0.15.1 \
            tqdm>=4.66.1 \
            numpy>=1.24.4 \
            tiktoken>=0.5.1 \
            wandb>=0.15.12

In [4]:
from fine_tune.train import train_model_with_samples, calc_loss_batch, calc_loss_loader, evaluate_model
from fine_tune.dataset import download_dataset, load_subset, train_test_split, create_dataloader
from fine_tune.dataset import InstructionDataset
from base_model.utils import text_to_token_ids, token_ids_to_text, generate
from fine_tune.utils import plot_losses, format_input
from base_model.config import GPT2Config
from base_model.model import GPTModel
from fine_tune.config import TrainingConfig
import tiktoken
import torch
import random

In [5]:
# create dataset and dataloader
config = TrainingConfig()
full_dataset = download_dataset(config)
sub_dataset = load_subset(full_dataset, config)

example = sub_dataset[random.randint(0,len(sub_dataset))]
formatted_ex = format_input(example)

print(f'Random Example:\n{formatted_ex}\n')

Data Loaded Successfully
Number of entries in dataset: 51760
Subset Data loaded successfully
Number of entries in subset dataset: 3000
Random Example:
Below is an instruction that describes a task, paired with an input that provides further context.

Write a response that appropriately completes the request.

### Instruction:
Provide two pieces of advice related to the given topic.

### Input:
Email etiquette

### Response:




In [6]:
# train test split
train_data, test_data, val_data = train_test_split(
    sub_dataset,config)

Train set size: 2550
Test set size: 300
Validation set size: 150


In [7]:
# dataloaders
tokenizer = tiktoken.get_encoding('gpt2')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

train_loader, test_loader, val_loader = create_dataloader(
    train_data, test_data, val_data,
    tokenizer, config, device=device)

for idx, (X, y) in enumerate(train_loader):
    print(f'Input Shape: {X.shape} | Target Shape: {y.shape}')
    break

Input Shape: torch.Size([8, 492]) | Target Shape: torch.Size([8, 492])


Loading our custom GPT2 with pretrained weights

In [8]:
from base_model.load_weights import load_gpt2_weights_to_model
config = GPT2Config()

# load model with pretrained weights
model= load_gpt2_weights_to_model(config)
model

Loading pre-trained GPT-2 model from Hugging Face...


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Initializing custom model...
Weights successfully loaded into custom model.


GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (emb_dropout): Dropout(p=0.1, inplace=False)
  (trf_blocks): ModuleList(
    (0-11): 12 x Block(
      (ln1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): CausalMultiHeadAttention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (res_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): MLP(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (gelu): GELU(approximate='tanh')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (final_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (out_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [9]:
formatted_ex

'Below is an instruction that describes a task, paired with an input that provides further context.\n\nWrite a response that appropriately completes the request.\n\n### Instruction:\nProvide two pieces of advice related to the given topic.\n\n### Input:\nEmail etiquette\n\n### Response:\n'

In [10]:
model.eval
input_ids = text_to_token_ids(
    formatted_ex, tokenizer)

out = model(input_ids)
# print(out)
print(out.size())

torch.Size([1, 61, 50257])


In [11]:
out_ids = generate(model=model,
                       idx=input_ids,
                       max_new_tokens=100,
                       context_size= config.context_length,
                       eos_id= 50256,
                       temp=0.8,
                       top_k=50)
text = token_ids_to_text(out_ids, tokenizer)
print(text)

Below is an instruction that describes a task, paired with an input that provides further context.

Write a response that appropriately completes the request.

### Instruction:
Provide two pieces of advice related to the given topic.

### Input:
Email etiquette

### Response:

This is the instruction that describes the given field in the current email.

### Content:

This is the instruction that describes the current form of response to an email.

### Description:

This is the response that describes a field you are trying to reply to.

### Example:

Write a reply.

### Email etiquette

This is the instruction that describes the current field in the current email.

### Content:

This is


In [12]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: aashu-0 (aashu-0-mnit) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [13]:
from fine_tune.train import train_model_with_samples, calc_loss_batch, calc_loss_loader, evaluate_model
from fine_tune.config import TrainingConfig
import time

train_config = TrainingConfig(
    start_context ="Write a short story",
    wandb_project = 'fine_tuning_gpt2_alpaca3k')

model.to(device)
start_time = time.time()
torch.manual_seed(123)
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=0.00005,
    weight_decay=0.1)

train_losses, val_losses, track_tokens_seen = train_model_with_samples(
    model,
    train_loader,
    val_loader,
    optimizer,
    start_context= val_data[0],
    device= device,
    tokenizer= tokenizer,
    config= train_config
)

end_time = time.time()

time_in_mins = (end_time - start_time)/60
print(f'Time taken to train: {time_in_mins:.2f} minutes')

Training model for 3 epochs with gradient accumlation every 4 steps

Epoch 1/3
Epoch: 1/3 | Step: 000005 | Tokens: 63,936
Train Loss: 2.1730 | Val Loss: 2.2209
Epoch: 1/3 | Step: 000010 | Tokens: 124,768
Train Loss: 2.0538 | Val Loss: 1.9908
Epoch: 1/3 | Step: 000015 | Tokens: 192,064
Train Loss: 1.8861 | Val Loss: 1.9377
Epoch: 1/3 | Step: 000020 | Tokens: 254,528
Train Loss: 1.8716 | Val Loss: 1.9061
Epoch: 1/3 | Step: 000025 | Tokens: 321,984
Train Loss: 1.8909 | Val Loss: 1.8901
Epoch: 1/3 | Step: 000030 | Tokens: 381,600
Train Loss: 1.7708 | Val Loss: 1.8800
Epoch: 1/3 | Step: 000035 | Tokens: 448,768
Train Loss: 1.9054 | Val Loss: 1.8715
Epoch: 1/3 | Step: 000040 | Tokens: 506,784
Train Loss: 1.7892 | Val Loss: 1.8629
Epoch: 1/3 | Step: 000045 | Tokens: 572,224
Train Loss: 1.7810 | Val Loss: 1.8550
Epoch: 1/3 | Step: 000050 | Tokens: 644,832
Train Loss: 1.8151 | Val Loss: 1.8461
Epoch: 1/3 | Step: 000055 | Tokens: 714,400
Train Loss: 1.8314 | Val Loss: 1.8415
Epoch: 1/3 | Step: 0

TypeError: argument 'text': 'dict' object cannot be converted to 'PyString'

In [ ]:
# need to fix this